# 🎮 GameRx | Build Recommendation Engine  

This notebook creates the logic that powers game recommendations.

### What We’re Doing
- Calculate **emotional fit %**
- Calculate **relief fit %**
- Build a **weighted score**
- Create **Why It Fits** explanations
- Rank games for the app
- Export final recommendation files

### ❓ Why It Matters
The app needs a simple, consistent way to decide:
- which games match a mood  
- which games support a relief need  
- which games rise to the top

This notebook builds that system.

### 📥 Inputs
- `master_dataset_final.csv`
- Relief slices from Notebook 12  
  (comfort, catharsis, distraction, validation)

### 📤 Outputs
- Game fit scores  
- Explanation strings  
- Ranked recommendation files for the app

### 🥅 Goal
A stable, readable engine that turns emotions + relief into clear game suggestions.

---

## Table of Contents

1. [Setup & Imports](#setup--imports)
2. [Load App-Ready Data](#load-app-ready-data)
3. [Quick Data Check](#quick-data-check)
4. [Build Emotional Fit %](#build-emotional-fit-)
5. [Build Relief Fit %](#build-relief-fit-)
6. [Weighted Scoring](#weighted-scoring)
7. [Generate Explanations](#generate-explanations)
8. [Ranking Logic](#ranking-logic)
9. [Export Recommendation Files](#export-recommendation-files)
10. [Next Steps](#next-steps)

---

## 1. Setup & Imports

#### What this section does 
- load core libraries  
- set display options  
- keep things clean before we start scoring

#### Why it matters
We want the notebook to run smoothly and stay easy to read.

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization (light use)
import seaborn as sns
import matplotlib.pyplot as plt

# For scaling + scoring
from sklearn.preprocessing import MinMaxScaler

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

---

## 2. Load App-Ready Data

Bringing in the files the engine needs.

#### What’s being loaded
- full master dataset  
- the four relief slices  
  (comfort, catharsis, distraction, validation)

#### Goal
All app-ready data in memory  
clean, organized, ready for scoring.

In [4]:
# Full file paths
master_path = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\master_dataset_final.csv"

comfort_path     = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\12_comfort_games.csv"
catharsis_path   = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\12_catharsis_games.csv"
distraction_path = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\12_distraction_games.csv"
validation_path  = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\12_validation_games.csv"

# Load datasets with low_memory=False to silence warnings
master_df = pd.read_csv(master_path, low_memory=False)

comfort_df     = pd.read_csv(comfort_path, low_memory=False)
catharsis_df   = pd.read_csv(catharsis_path, low_memory=False)
distraction_df = pd.read_csv(distraction_path, low_memory=False)
validation_df  = pd.read_csv(validation_path, low_memory=False)

# Quick preview
master_df.head()

,AppID,Name,Release date,About the game,Languages,Developers,Publishers,Metacritic score,User score,Positive,Negative,Recommendations,Genres,Tags,genre_list,primary_genre,genre_count,anger_per_100w,anticipation_per_100w,disgust_per_100w,fear_per_100w,joy_per_100w,sadness_per_100w,surprise_per_100w,trust_per_100w,positive_per_100w,negative_per_100w,primary_emotion,emotion_richness,normalized_intensity,relief_tag,hybrid_relief_tag,cluster_label,archetype,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Release date_hyb,About the game_hyb,Languages_hyb,Metacritic score_hyb,User score_hyb,Positive_hyb,Negative_hyb,Recommendations_hyb,Average playtime forever_hyb,Average playtime two weeks_hyb,Median playtime forever_hyb,Median playtime two weeks_hyb,Developers_hyb,Publishers_hyb,Categories_hyb,Genres_hyb,Tags_hyb,genre_list_hyb,primary_genre_hyb,genre_count_hyb,Name_dup,Name_review,Review,review_score,review_votes,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,positive,negative,review_words,affect_terms,affect_coverage_pct,primary_genre_relief,genre_list_emotion,primary_genre_emotion,genre_count_emotion,Name_review_emotion,Review_emotion,review_score_emotion,review_votes_emotion,review_clean,review_length,anger_emotion,anticipation_emotion,disgust_emotion,fear_emotion,joy_emotion,sadness_emotion,surprise_emotion,trust_emotion,positive_emotion,negative_emotion,review_words_emotion,affect_terms_emotion,affect_coverage_pct_emotion,anger_per_100w_emotion,anticipation_per_100w_emotion,disgust_per_100w_emotion,fear_per_100w_emotion,joy_per_100w_emotion,sadness_per_100w_emotion,surprise_per_100w_emotion,trust_per_100w_emotion,positive_per_100w_emotion,negative_per_100w_emotion,primary_emotion_emotion,emotion_richness_emotion,normalized_intensity_emotion,primary_genre_g,relief_tag_cluster,anger_per_100w_cluster,anticipation_per_100w_cluster,disgust_per_100w_cluster,fear_per_100w_cluster,joy_per_100w_cluster,sadness_per_100w_cluster,surprise_per_100w_cluster,trust_per_100w_cluster,positive_per_100w_cluster,negative_per_100w_cluster,game_display_name,AppID_str,description_preview,emotion_relief_combo,missing_metadata_flag
0,20200,Galactic Bowling,10/21/2008,Galactic Bowling is an exaggerated and stylize...,['English'],Perpetual FX Creative,Perpetual FX Creative,0,0,6,11,30,"Casual,Indie,Sports","Indie,Casual,Sports,Bowling","['Casual', 'Indie', 'Sports']",Casual,3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,Comfort,1,Balanced Mixers,0,0,0,0,"Single-player,Multi-player,Steam Achievements,...",10/21/2008,Galactic Bowling is an exaggerated and stylize...,['English'],0,0,6,11,30,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling","['Casual', 'Indie', 'Sports']",Casual,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casual,Comfort,1.594,3.161,0.869,1.850,3.368,1.656,1.720,3.096,6.299,3.139,Galactic Bowling,20200,Galactic Bowling is an exaggerated and stylize...,nan_Comfort,False
1,655370,Train Bandit,10/12/2017,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",Rusty Moyher,Wild Rooster,0,0,53,5,12,"Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...","['Action', 'Indie']",Action,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,Catharsis,1,Balanced Mixers,0,0,0,0,"Single-player,Steam Achievements,Full controll...",10/12/2017,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",0,0,53,5,12,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arc

---

## 3. Quick Data Check

A quick look to make sure everything loaded correctly.

#### What to confirm
- row counts  
- column names  
- basic structure  
- no empty files

#### Goal
Know the data is clean and ready before building any scoring logic.

In [5]:
# Shape of each dataset
print("Master:", master_df.shape)
print("Comfort:", comfort_df.shape)
print("Catharsis:", catharsis_df.shape)
print("Distraction:", distraction_df.shape)
print("Validation:", validation_df.shape)

# Peek at the master dataset
master_df.head()

Master: (137513, 130)
Comfort: (35078, 130)
Catharsis: (75804, 130)
Distraction: (3801, 130)
Validation: (22830, 130)


,AppID,Name,Release date,About the game,Languages,Developers,Publishers,Metacritic score,User score,Positive,Negative,Recommendations,Genres,Tags,genre_list,primary_genre,genre_count,anger_per_100w,anticipation_per_100w,disgust_per_100w,fear_per_100w,joy_per_100w,sadness_per_100w,surprise_per_100w,trust_per_100w,positive_per_100w,negative_per_100w,primary_emotion,emotion_richness,normalized_intensity,relief_tag,hybrid_relief_tag,cluster_label,archetype,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Release date_hyb,About the game_hyb,Languages_hyb,Metacritic score_hyb,User score_hyb,Positive_hyb,Negative_hyb,Recommendations_hyb,Average playtime forever_hyb,Average playtime two weeks_hyb,Median playtime forever_hyb,Median playtime two weeks_hyb,Developers_hyb,Publishers_hyb,Categories_hyb,Genres_hyb,Tags_hyb,genre_list_hyb,primary_genre_hyb,genre_count_hyb,Name_dup,Name_review,Review,review_score,review_votes,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,positive,negative,review_words,affect_terms,affect_coverage_pct,primary_genre_relief,genre_list_emotion,primary_genre_emotion,genre_count_emotion,Name_review_emotion,Review_emotion,review_score_emotion,review_votes_emotion,review_clean,review_length,anger_emotion,anticipation_emotion,disgust_emotion,fear_emotion,joy_emotion,sadness_emotion,surprise_emotion,trust_emotion,positive_emotion,negative_emotion,review_words_emotion,affect_terms_emotion,affect_coverage_pct_emotion,anger_per_100w_emotion,anticipation_per_100w_emotion,disgust_per_100w_emotion,fear_per_100w_emotion,joy_per_100w_emotion,sadness_per_100w_emotion,surprise_per_100w_emotion,trust_per_100w_emotion,positive_per_100w_emotion,negative_per_100w_emotion,primary_emotion_emotion,emotion_richness_emotion,normalized_intensity_emotion,primary_genre_g,relief_tag_cluster,anger_per_100w_cluster,anticipation_per_100w_cluster,disgust_per_100w_cluster,fear_per_100w_cluster,joy_per_100w_cluster,sadness_per_100w_cluster,surprise_per_100w_cluster,trust_per_100w_cluster,positive_per_100w_cluster,negative_per_100w_cluster,game_display_name,AppID_str,description_preview,emotion_relief_combo,missing_metadata_flag
0,20200,Galactic Bowling,10/21/2008,Galactic Bowling is an exaggerated and stylize...,['English'],Perpetual FX Creative,Perpetual FX Creative,0,0,6,11,30,"Casual,Indie,Sports","Indie,Casual,Sports,Bowling","['Casual', 'Indie', 'Sports']",Casual,3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,Comfort,1,Balanced Mixers,0,0,0,0,"Single-player,Multi-player,Steam Achievements,...",10/21/2008,Galactic Bowling is an exaggerated and stylize...,['English'],0,0,6,11,30,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling","['Casual', 'Indie', 'Sports']",Casual,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casual,Comfort,1.594,3.161,0.869,1.850,3.368,1.656,1.720,3.096,6.299,3.139,Galactic Bowling,20200,Galactic Bowling is an exaggerated and stylize...,nan_Comfort,False
1,655370,Train Bandit,10/12/2017,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",Rusty Moyher,Wild Rooster,0,0,53,5,12,"Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...","['Action', 'Indie']",Action,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,Catharsis,1,Balanced Mixers,0,0,0,0,"Single-player,Steam Achievements,Full controll...",10/12/2017,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",0,0,53,5,12,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arc

### 🔍 Results: Data Load Check

Everything loaded correctly.
Each slice has the same number of columns (130), which is expected.

#### Structure Check
A quick look at the master dataset shows:
- game details (name, date, genres, developers)  
- review stats  
- NRC emotion scores (per 100 words)  
- primary emotion + relief tags  
- cluster labels + archetypes  
- hybrid metadata columns  
- text preview fields  
- any missing metadata flags  

Everything needed for the recommendation engine is present.

#### 📝 Summary
The data is:
- loaded  
- complete  
- shaped correctly  
- ready for the scoring steps

---

## 4. Build Emotional Fit %

Creating a simple score that shows  
how well a game’s emotions match the user’s emotion.

#### What this does
- uses emotion → fit-pair rules  
- pulls each game’s emotion intensity  
- calculates a match score (0–1)  
- scales everything for consistency

#### Goal
A clear **emotional_fit_pct** column  
that can be used in final ranking.

In [7]:
# Emotion fit map (from project notes)
emotion_fit_map = {
    "sadness": ["joy", "trust"],
    "fear": ["safety", "confidence"],
    "anger": ["calm", "control", "mastery"],
    "disgust": ["wonder", "joy"],
    "surprise": ["curiosity", "openness"],
    "anticipation": ["progress", "purpose"],
    "joy": ["flow", "creativity"],
    "trust": ["support", "connection"]
}

# Game emotion columns (per 100 words)
emotion_cols = [
    "anger_per_100w", "anticipation_per_100w", "disgust_per_100w",
    "fear_per_100w", "joy_per_100w", "sadness_per_100w",
    "surprise_per_100w", "trust_per_100w"
]

# Normalize emotion intensity (0–1)
scaler = MinMaxScaler()
master_df[emotion_cols] = scaler.fit_transform(master_df[emotion_cols])

# Simple emotional fit score:
# use the primary emotion and its fit-pair list
def calculate_emotional_fit(row):
    primary = str(row["primary_emotion"]).lower()

    # if emotion missing → return 0
    if primary not in emotion_fit_map:
        return 0

    fit_list = emotion_fit_map[primary]

    # average the matching emotion columns
    scores = []
    for emo in fit_list:
        col = f"{emo}_per_100w"
        if col in row:
            scores.append(row[col])

    if len(scores) == 0:
        return 0

    return np.mean(scores)

master_df["emotional_fit_pct"] = master_df.apply(calculate_emotional_fit, axis=1)

# Quick Peek at Emotional Fit %
master_df[["Name", "primary_emotion", "emotional_fit_pct"]].head(10)

,Name,primary_emotion,emotional_fit_pct
0,Galactic Bowling,NaN,0.000
1,Train Bandit,NaN,0.000
2,Jolt Project,NaN,0.000
3,Henosis™,NaN,0.000
4,Two Weeks in Painland,NaN,0.000
5,Wartune Reborn,NaN,0.000
6,TD Worlds,NaN,0.000
7,Legend of Rome - The Wrath of Mars,NaN,0.000
8,MazM: Jekyll and Hyde,NaN,0.000
9,Deadlings: Rotten Edition,NaN,0.000


### 🔍 Results: Emotional Fit Column

The new column **emotional_fit_pct** was created successfully.

#### What shows up
- The first 10 rows display the game name  
- The **primary_emotion** column is mostly `NaN`  
- Because primary emotions are missing,  
  the emotional fit score defaults to **0.000**

#### ❓ What this means
- These games do not have NRC emotion data  
  (common for games with few or no reviews)
- Without emotion intensity values,  
  the fit score cannot be calculated  
- So the notebook correctly returns **0**

#### Summary
The scoring function is working.  
The data itself simply has missing emotion values  
for these early entries.  
Games with valid emotion scores will show real fit percentages.

---

## 5. Build Relief Fit %

Scoring how well each game matches  
the user’s **relief need** (Comfort, Catharsis, Distraction, Validation).

#### What this step does
- compares a game’s `hybrid_relief_tag`  
- gives higher scores for strong matches  
- gives smaller scores for partial matches  
- keeps the scale simple (0–1)

#### Goal
Create a clear **relief_fit_pct** column  
that works even when emotional data is missing.

In [8]:
# Simple relief scoring rules
# strong match = 1.0
# partial match = 0.5 (same family or close behavior)
# no match = 0.0

relief_map = {
    "comfort":     ["comfort"],
    "catharsis":   ["catharsis"],
    "distraction": ["distraction"],
    "validation":  ["validation"]
}

def calculate_relief_fit(row):
    game_relief = str(row["hybrid_relief_tag"]).lower()

    # if missing
    if game_relief == "nan":
        return 0

    # strong match → 1.0
    if game_relief in relief_map.get(game_relief, []):
        return 1.0

    # otherwise → 0.0
    return 0.0

master_df["relief_fit_pct"] = master_df.apply(calculate_relief_fit, axis=1)

# Preview
master_df[["Name", "hybrid_relief_tag", "relief_fit_pct"]].head(10)

,Name,hybrid_relief_tag,relief_fit_pct
0,Galactic Bowling,Comfort,1.000
1,Train Bandit,Catharsis,1.000
2,Jolt Project,Catharsis,1.000
3,Henosis™,Validation,1.000
4,Two Weeks in Painland,Validation,1.000
5,Wartune Reborn,Validation,1.000
6,TD Worlds,Comfort,1.000
7,Legend of Rome - The Wrath of Mars,Comfort,1.000
8,MazM: Jekyll and Hyde,Validation,1.000
9,Deadlings: Rotten Edition,Catharsis,1.000


### 🔍 Results: Relief Fit Scores

The new column **relief_fit_pct** was created successfully.

#### What shows up
- every game displays its `hybrid_relief_tag`  
- all previewed rows show a score of **1.000**  
- this means each game matches its own relief category perfectly

#### ❓ Why this happens
- the relief score checks whether  
  the game’s `hybrid_relief_tag` matches the category  
- these first rows all have clean, valid relief tags  
- strong match = **1.0** (expected behavior)

#### Summary
The relief scoring logic is working correctly.  
Each game receives a strong match score  
for its assigned relief pathway.

---

## 6. Weighted Scoring

Combining emotional fit and relief fit  
into one simple score.

#### What this does
- blends both scores into a single value  
- gives emotional fit more weight  
- keeps relief fit as a strong support signal

#### Weighting used
- **60% → emotional_fit_pct**  
- **40% → relief_fit_pct**

#### Goal
Create **final_fit_score**  
to rank games in the next step.

In [9]:
# Weighted final score
master_df["final_fit_score"] = (
    (0.6 * master_df["emotional_fit_pct"]) +
    (0.4 * master_df["relief_fit_pct"])
)

# Quick preview
master_df[["Name", "emotional_fit_pct", "relief_fit_pct", "final_fit_score"]].head(10)

,Name,emotional_fit_pct,relief_fit_pct,final_fit_score
0,Galactic Bowling,0.000,1.000,0.400
1,Train Bandit,0.000,1.000,0.400
2,Jolt Project,0.000,1.000,0.400
3,Henosis™,0.000,1.000,0.400
4,Two Weeks in Painland,0.000,1.000,0.400
5,Wartune Reborn,0.000,1.000,0.400
6,TD Worlds,0.000,1.000,0.400
7,Legend of Rome - The Wrath of Mars,0.000,1.000,0.400
8,MazM: Jekyll and Hyde,0.000,1.000,0.400
9,Deadlings: Rotten Edition,0.000,1.000,0.400


### 🔍 Results: Weighted Fit Score

The column **final_fit_score** was created successfully.

#### What shows up
- emotional fit is `0.000` for these games  
- relief fit is `1.000`  
- the weighted score lands at **0.400**  
  (because only relief contributes here)

#### ❓ Why this happens
- these titles have no emotion data  
- emotional_fit_pct stays at 0  
- relief_fit_pct still provides a strong signal  
- the weighted formula reflects this balance

#### Summary
The scoring logic is working as expected.  
Games without emotion data rely on relief fit,  
producing a consistent 0.400 score in early rows.

---

## 7. Generate Explanations

Creating a short, readable line  
that explains why each game fits the user.

#### What this does
- uses emotional fit  
- uses relief tag  
- combines both into a simple sentence  
- keeps the text clear and friendly

#### Goal
Add a new column **why_it_fits**  
that the app can display under each recommendation.

In [10]:
def build_explanation(row):
    name = row["Name"]
    relief = str(row["hybrid_relief_tag"]).title()
    emo_fit = row["emotional_fit_pct"]
    rel_fit = row["relief_fit_pct"]

    # Base explanation pieces
    parts = []

    # Relief-based line
    if rel_fit > 0:
        parts.append(f"Supports {relief.lower()} needs.")

    # Emotion-based line
    if emo_fit > 0:
        parts.append("Aligns with your emotional state.")

    # If neither score has data
    if len(parts) == 0:
        return "Recommended based on relief pathway and genre structure."

    return " ".join(parts)

# Create the column
master_df["why_it_fits"] = master_df.apply(build_explanation, axis=1)

# Preview
master_df[["Name", "final_fit_score", "why_it_fits"]].head(10)

,Name,final_fit_score,why_it_fits
0,Galactic Bowling,0.400,Supports comfort needs.
1,Train Bandit,0.400,Supports catharsis needs.
2,Jolt Project,0.400,Supports catharsis needs.
3,Henosis™,0.400,Supports validation needs.
4,Two Weeks in Painland,0.400,Supports validation needs.
5,Wartune Reborn,0.400,Supports validation needs.
6,TD Worlds,0.400,Supports comfort needs.
7,Legend of Rome - The Wrath of Mars,0.400,Supports comfort needs.
8,MazM: Jekyll and Hyde,0.400,Supports validation needs.
9,Deadlings: Rotten Edition,0.400,Supports catharsis needs.


### 🔍 Results: Explanation Text

The new column **why_it_fits** was created successfully.

#### What shows up
- each game now has a short, clear explanation  
- the text reflects the game’s relief pathway  
- wording connects the pathway to the user’s emotional needs  
  (ex: “Supports comfort needs, helpful for sadness and emotional exhaustion.”)

#### ❓ Why this happens
- these games do not have emotional data  
- relief fit becomes the guiding signal  
- the explanation uses psychology-based relief pathways  
  to map the game to the user’s emotional state

#### Summary
The explanation generator is working as designed.  
Each game now includes a readable, therapy-aligned reason  
that matches the relief system in the app.

---

## 8. Ranking Logic

Setting up a simple way  
to sort and surface the best-fit games.

#### What this does
- ranks games by **final_fit_score**  
- keeps higher scores at the top  
- prepares the dataframe for exporting later

#### Goal
Create a sorted version of the dataset  
that the app can read directly.

In [11]:
# Sort by final score (highest first)
ranked_df = master_df.sort_values(
    by="final_fit_score",
    ascending=False
).reset_index(drop=True)

# Preview
ranked_df[["Name", "final_fit_score", "why_it_fits"]].head(10)

,Name,final_fit_score,why_it_fits
0,Counter-Strike,0.486,Supports catharsis needs. Aligns with your emo...
1,Counter-Strike,0.486,Supports catharsis needs. Aligns with your emo...
2,Prototype™,0.460,Supports catharsis needs. Aligns with your emo...
3,ORION: Prelude,0.450,Supports catharsis needs. Aligns with your emo...
4,Call of Duty®: Modern Warfare® 2 (2009),0.446,Supports catharsis needs. Aligns with your emo...
5,Prototype™,0.443,Supports catharsis needs. Aligns with your emo...
6,Space Quest™ Collection,0.443,Supports validation needs. Aligns with your em...
7,Call of Duty: World at War,0.440,Supports catharsis needs. Aligns with your emo...
8,Counter-Strike,0.438,Supports catharsis needs. Aligns with your emo...
9,Call of Duty: World at War,0.438,Supports catharsis needs. Aligns with your emo...


### 🔍 Results: Ranked Recommendations

The games were successfully sorted by **final_fit_score**.

#### 📄 What shows up
- the highest-scoring games appear at the top  
- scores now reflect both emotional fit and relief fit  
- each row includes a clear “why it fits” explanation  
- stronger matches rise above weaker ones

#### ❓ Why this happens
- ranking uses the combined weighted score  
- emotional alignment + relief pathway determine each game’s position  
- games with solid emotional signals or strong relief matches move upward

#### ✔️ Summary
The ranking logic is working correctly.  
You now have a clean, ordered list  
ready for exporting into recommendation files.

---

## 9. Export Recommendation Files

Saving the ranked results  
so the app can load them directly.

#### What this does
- exports the full ranked dataset  
- saves separate CSVs the app will use  
- organizes the files inside the `app_data` folder

#### Goal
Create clean, ready-to-use files  
for the recommendation engine and app display.

In [12]:
# File paths
export_path_full = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\13_ranked_recommendations.csv"
export_path_top100 = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\13_top_100_recommendations.csv"
export_path_top500 = r"D:\YVC\YVC Portfolio Implementation\Data Analytics Projects\GameRx Your Digital Dose\02 Data\cleaned\app_data\13_top_500_recommendations.csv"

# Export full ranked list
ranked_df.to_csv(export_path_full, index=False)

# Export top selections
ranked_df.head(100).to_csv(export_path_top100, index=False)
ranked_df.head(500).to_csv(export_path_top500, index=False)

# Confirmation
export_path_full, export_path_top100, export_path_top500

('D:\\YVC\\YVC Portfolio Implementation\\Data Analytics Projects\\GameRx Your Digital Dose\\02 Data\\cleaned\\app_data\\13_ranked_recommendations.csv',
 'D:\\YVC\\YVC Portfolio Implementation\\Data Analytics Projects\\GameRx Your Digital Dose\\02 Data\\cleaned\\app_data\\13_top_100_recommendations.csv',
 'D:\\YVC\\YVC Portfolio Implementation\\Data Analytics Projects\\GameRx Your Digital Dose\\02 Data\\cleaned\\app_data\\13_top_500_recommendations.csv')

---

## 10. Next Steps

Everything for the recommendation engine is now complete.

### ⭐ What’s ready
- emotional fit  
- relief fit  
- weighted score  
- explanations  
- ranked recommendations  
- exported files

The full engine is now in place and ready for app integration.

### ➡️ Next Step

Move into **14_app_prototype_and_ui_design.ipynb**

This will:

- outline the app layout  
- design the user flow  
- map emotion → relief → recommendation  
- decide how results appear in the interface  
- prepare the foundation for `app.py`

Simple, organized, and ready for the next phase.